# **Import các thư viện cần thiết**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import os
import json
import numpy as np
from PIL import Image
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from collections import Counter

# **Tải mô hình VGG16 đã được huấn luyện trước**:
=> Để trích xuất feature maps làm đầu vào cho RPN. Ở đây chỉ lấy phần đặc trưng (features), bỏ qua các lớp fully connected.

## Một số thông tin về trích xuất đặc trưng bằng VGG 16

- VGG16 có kiến trúc bao gồm nhiều lớp tích chập và pooling giúp nó tạo ra các bản đồ đặc trưng (feature maps) từ ảnh. Các lớp này có khả năng trích xuất đặc trưng theo nhiều cấp độ khác nhau, từ các đặc trưng đơn giản (ví dụ như cạnh viền) đến các đặc trưng phức tạp (ví dụ như khuôn mặt, đồ vật).


In [ ]:
VGG16_model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True)

# **Xây dựng mạng Backbone**

Backbone sẽ nhận đầu vào là ảnh và tạo ra feature map (bản đồ đặc trưng) làm đầu vào cho các lớp tiếp theo.
Ở đây sẽ chỉ lấy các feature và bỏ qua fully connected layers (dùng cho classification)



In [ ]:
CNN_back_bone = nn.Sequential(*list(VGG16_model.features.children())[:-1])
#list(VGG16_model.features.children(): chuyển các lớp con thành một danh sách
#[:-1]: bỏ lớp cuối (lớp pooling)

In [ ]:
anchor_ratio = [[1, 1], [1, 2], [2, 1]] #tỉ lệ các anchor
anchor_size = [128, 256, 512]

class RPN(nn.Module):
    def __init__(self,input_channels, k=9):
        #Convolution Layers
        super(RPN,self).__init__()
        self.conv = nn.Conv2d(input_channels, 512, 3, 1, 1) # 512 filters, kernel 3X3
        self.cls = nn.Conv2d(512, k, 1) #=0 -> k chứa vật thể / =1 -> chứa vật thể
        self.reg = nn.Conv2d(512, 4 * k, 1) #


    def forward(self, x):
        '''
            output: cls:[B, 9, H, W], reg:[B, 36, H, W]
        '''
        x = F.relu(self.conv(x))
        cls = F.sigmoid(self.cls(x))
        reg = self.reg(x)
        return cls, reg


In [ ]:
class RPNLoss(nn.Module):
    def __init__(self):
        super(RPNLoss,self).__init__()
        self.cls_loss = nn.BCELoss()
        self.reg_loss = nn.SmoothL1Loss(reduction='mean')
        self.lam_reg = 10
        self.k = 9

    def forward(self, cls_pred, cls_gt, reg_pred, reg_gt,):
        '''
            cls_shape = [B, T, 1] T = 256
            reg_shape = [B, N, M, 4] N*M = 2400
        '''
        N_cls = cls_gt.shape[0]

        cls_loss = self.cls_loss(cls_pred, cls_gt)


        reg_pred = reg_pred.permute(0, 2, 3, 1)
        size_batch_reg = reg_pred.shape[0]
        reg_pred = reg_pred.reshape(size_batch_reg, reg_pred.shape[1]* reg_pred.shape[2], 4*self.k)

        reg_gt = reg_gt.permute(0, 2, 3, 1)
        reg_gt = reg_gt.reshape(size_batch_reg, reg_gt.shape[1]* reg_gt.shape[2], 4*self.k)
        pos_gt = reg_gt > 0
        # pos_count = torch.zeros_like(reg_gt)
        # pos_count[pos_gt] = 1
        reg_loss = torch.sum(self.reg_loss(reg_pred[pos_gt], reg_gt[pos_gt]))
        loss = cls_loss + self.lam_reg * reg_loss * 4
        return loss

In [ ]:
loss_fn = RPNLoss()

In [ ]:
!kaggle datasets download -d zaraks/pascal-voc-2007

In [ ]:
!unzip pascal-voc-2007.zip -d pascal-voc-2007

In [ ]:
size_img = [600, 600]

In [ ]:
def convert_coord(coord, scale=16):
    return coord*scale

def get_feature_size(size, scale=16):
    return [size[-2]//scale, size[-1]//scale]

def convert_box(bbox, btype='conner'):
    bboxes = bbox.clone()
    if (btype == 'conner'):
        bboxes[...,:2] -= bboxes[..., 2:]/2
        bboxes[...,2:] += bboxes[..., :2]
        return bboxes

    elif (btype == 'center'):
        bboxes[:, :2] += bboxes[:, 2:]
        bboxes[:, :2] /= 2
        bboxes[:, 2:] -= bboxes[:, :2]
        bboxes[:, 2:] *= 2
        return bboxes

def IoU(box1, box2):
    box1 = convert_box(box1)
    box2 = convert_box(box2)
    U_x_min = torch.min(box1[...,0], box2[...,0])
    U_y_min = torch.min(box1[...,1], box2[...,1])
    U_x_max = torch.max(box1[...,2], box2[...,2])
    U_y_max = torch.max(box1[...,3], box2[...,3])
    U_area = (U_x_max - U_x_min) * (U_y_max - U_y_min)
    I_x_min = torch.max(box1[...,0], box2[...,0])
    I_y_min = torch.max(box1[...,1], box2[...,1])
    I_x_max = torch.min(box1[...,2], box2[...,2])
    I_y_max = torch.min(box1[...,3], box2[...,3])
    I_area = (I_x_max - I_x_min) * (I_y_max - I_y_min)
    I_area[I_x_min >= I_x_max] = 0
    I_area[I_y_min >= I_y_max] = 0
    return I_area / U_area

class Pascal_Dataset(object):
    def __init__(self, json_file, dir_img, transform=None, k=9):
        with open(json_file, 'r') as file:
            self.json_file = json.load(file)
        self.dir_img = dir_img
        self.imgs_name, self.imgs_size = self.get_img_name()
        self.bboxes_img= self.get_bboxes()
        self.transform = transform
        self.k = k

    def __len__(self):
        return len(self.json_file['images'])

    def __getitem__(self, idx):
        '''
            input: idx
            output: img:tensor, bbox:tensor
        '''
        img_path = os.path.join(self.dir_img, self.imgs_name[idx])
        img = Image.open(img_path)
        bboxes = self.bboxes_img[idx]
        bboxes = torch.Tensor(bboxes)
        if self.transform:
            img, bboxes = self.transform(img, bboxes)
        bboxes[:, :2] += bboxes[:, 2:]/2
        cls_gt, reg_gt = self.build_gt(img, bboxes)
        return img, cls_gt, reg_gt

    def get_img_name(self):
        imgs = self.json_file['images']
        img_name = {img['id']:img['file_name'] for img in imgs}
        img_size = {img['id']:[img['height'], img['width']] for img in imgs}
        return list(img_name.values()), img_size

    def get_bboxes(self):
        imgs = self.json_file['annotations']
        bboxes = {}
        for bbox in imgs:
            if bbox['image_id'] not in bboxes:
                bboxes[bbox['image_id']] = []

            bboxes[bbox['image_id']].append(self.resize(bbox['image_id'], bbox['bbox']))
        return list(bboxes.values())

    def resize(self, img_id, bbox):
        size = self.imgs_size[img_id]
        return [bbox[1]/size[0]*size_img[0], bbox[0]/size[1]*size_img[1], bbox[3]/size[0]*size_img[0], bbox[2]/size[1]*size_img[1]]

    def build_gt(self, img, bboxes, pos_thresh=0.5, neg_thresh=0.3):
        '''
            input: img:tensor, bboxes:tensor
            output: cls_gt:tensor, reg_gt:tensor
        '''
        feature_size = get_feature_size(img.shape)
        cls_gt = torch.zeros(1*self.k, feature_size[0], feature_size[1])
        reg_gt = torch.zeros(feature_size[0], feature_size[1], self.k, 4)
        bbox_gt = torch.zeros(feature_size[0], feature_size[1], self.k, 4)
        frame_anchor = []
        for scale in anchor_size:
            for ratio in anchor_ratio:
                frame_anchor.append([scale*ratio[0], scale*ratio[1]])
        for i in range(feature_size[0]):
            for j in range(feature_size[1]):
                bbox_gt[i, j] = torch.torch.Tensor([[convert_coord(i), convert_coord(j),frame_anchor[k][0], frame_anchor[k][1]] for k in range(self.k)])

        for bbox in bboxes:
            IoU_value = IoU(bbox_gt, bbox)
            index_pos = torch.argwhere(IoU_value > pos_thresh)
            cls_gt[index_pos[:, 2], index_pos[:, 0], index_pos[:, 1]] = 1
            reg_gt[index_pos[:, 0], index_pos[:, 1], index_pos[:, 2]] = self.build_offset(bbox_gt[index_pos[:, 0], index_pos[:, 1], index_pos[:, 2]], bbox)
        return cls_gt, (torch.flatten(reg_gt, start_dim=2)).permute(2, 0, 1)

    def build_offset(self, anchor, bbox):
        '''
            input: anchor:tensor, bbox:tensor
            output: offset:tensor
        '''
        t = torch.zeros(anchor.shape)
        t[...,0] = (bbox[0] - anchor[...,0])/anchor[...,2]
        t[...,1] = (bbox[1] - anchor[...,1])/anchor[...,3]
        t[...,2] = torch.log(bbox[2]/anchor[...,2])
        t[...,3] = torch.log(bbox[3]/anchor[...,3])
        return t



In [ ]:
class Compose():
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, bboxes):
        for t in self.transforms:
            img, bboxes = t(img), bboxes
        return img, bboxes

In [ ]:
seed = 125 # used seed: 123, 124
torch.manual_seed(seed)

LEARNING_RATE = 1e-3
DEVICE = "cuda" if torch.cuda.is_available else "cpu"
BATCH_SIZE = 32
WEIGHT_DECAY = 0.0005
EPOCHS = 30
NUM_WORKERS = 2
PIN_MEMORY = True
LOAD_MODEL = True
LOAD_MODEL_FILE = "f-RPN_20_epochs.pth"
IMG_DIR = '/content/pascal-voc-2007/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/JPEGImages'

In [ ]:
transform = Compose([transforms.Resize(size_img), transforms.ToTensor()])

In [ ]:
CNN_back_bone = CNN_back_bone.to(DEVICE)


In [ ]:

def train_fn(train_loader, model, optimizer, loss_fn):
    loop = tqdm(train_loader, leave=True)
    mean_loss = []
    CNN_back_bone.eval()
    for batch_idx, (x, cls_gt, reg_gt) in enumerate(loop):
        x, cls_gt, reg_gt = x.to(DEVICE), cls_gt.to(DEVICE), reg_gt.to(DEVICE)
        with torch.no_grad():
            feature_map = CNN_back_bone(x)
        cls_pred, reg_pred = model(feature_map)
        batch_cls_pred = torch.zeros((cls_gt.shape[0], 256, 1)).to(DEVICE)
        batch_cls_gt = torch.zeros((cls_gt.shape[0], 256, 1)).to(DEVICE)
        for i in range(cls_pred.shape[0]):
            mini_cls_pred = cls_pred[i].reshape(-1, 1)
            mini_cls_gt = cls_gt[i].reshape(-1, 1)
            pos_gt = torch.where(mini_cls_gt == 1)[0]
            neg_gt = torch.where(mini_cls_gt == 0)[0]
            nums = min(128, pos_gt.size()[0])
            pos_gt_size = pos_gt.size()[0]
            if pos_gt_size > 0:
                index = torch.cat((pos_gt[torch.randint(0, pos_gt.size()[0], (nums,))],
                                neg_gt[torch.randint(0, neg_gt.size()[0], (256-nums, ))]))
                batch_cls_pred[i] = mini_cls_pred[index]
                batch_cls_gt[i] = mini_cls_gt[index]

        loss = loss_fn(batch_cls_pred, batch_cls_gt, reg_pred, reg_gt)

        optimizer.zero_grad()
        loss.backward()
        mean_loss.append(loss.item())
        optimizer.step()

        #Update the progress bar
        loop.set_postfix(loss = loss.item())
    print(f'Mean loss was {sum(mean_loss)/len(mean_loss)}')


In [ ]:
def load_checkpoint(checkpoint_path, model, optimizer):
    # Load the checkpoint from the file
    # checkpoint = torch.load(checkpoint_path) # This line is incorrect. We've already loaded it

    # checkpoint_path is already the loaded checkpoint dictionary. Rename for clarity
    checkpoint = checkpoint_path

    # Load the model state_dict and optimizer state_dict from the checkpoint
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
model = RPN(512).to(DEVICE)

optimizer = optim.Adam(
    model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
)
loss_fn = RPNLoss()

if LOAD_MODEL:
    load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)

# if LOAD_MODEL:
#     load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)

FILE_TRAIN = '/content/pascal-voc-2007/pascal_voc/PASCAL_VOC/pascal_train2007.json'  # Thay thế bằng tên tệp JSON của bạn
FILE_TEST = '/content/pascal-voc-2007/pascal_voc/PASCAL_VOC/pascal_test2007.json'  # Thay thế bằng tên tệp JSON của bạn
DIR_IMG_TEST = '/content/pascal-voc-2007/VOCtest_06-Nov-2007/VOCdevkit/VOC2007/JPEGImages'

train_dataset = Pascal_Dataset(
    FILE_TRAIN,
    IMG_DIR,
    transform=transform
)

test_dataset = Pascal_Dataset(
    FILE_TEST,
    DIR_IMG_TEST,
    transform=transform
)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=True,
    drop_last=False,
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=True,
    drop_last=True,
)

In [ ]:
for epoch in range(EPOCHS):
    # pred_boxes, target_boxes = get_bboxes(
    #     train_loader, model, iou_threshold=0.5, threshold=0.4
    # )
    # mean_avg_prec = mean_average_precision(
    #     pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint"
    # )
    # print(f"Train mAP: {mean_avg_prec}")
    print(f'epoch: {epoch}')
    train_fn(train_loader, model, optimizer, loss_fn)

In [ ]:
torch.save({'model_state_dict':model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(),}, 'd-RPN-20_epochs.pth')

In [ ]:
x, cls_gt, reg_gt = train_dataset[1]
x = x.unsqueeze(0).to(DEVICE)
cls_gt = cls_gt.unsqueeze(0).to(DEVICE)
reg_gt = reg_gt.unsqueeze(0).to(DEVICE)
t = CNN_back_bone(x)
cls_pred, reg_pred = model(t)

In [ ]:
torch.argwhere(cls_pred[cls_gt > 0] > 0.6).shape[0] / torch.argwhere(cls_pred > 0.6).shape[0]


In [ ]:
def evalue(test_loader, model):
    loop = tqdm(test_loader, leave=True)
    CNN_back_bone.eval()
    for test_idx, (x, cls_gt, reg_gt) in enumerate(loop):
        with torch.no_grad():
            feature_map =  CNN_back_bone(x)
